In [1]:
import pandas as pd
import numpy as np
import warnings
from Dataset_functions import *
# Suppress all warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
premier_league_season17_19 = prepare_dataframe('17_18_eng.csv', '2017-2018')
premier_league_season18_19 = prepare_dataframe('18_19_eng.csv', '2018-2019')
premier_league_season19_20 = prepare_dataframe('19_20_eng.csv', '2019-2020')
premier_league_season20_21 = prepare_dataframe('20_21_eng.csv', '2020-2021')
premier_league_season21_22 = prepare_dataframe('21_22_eng.csv', '2021-2022')
premier_league_season22_23 = prepare_dataframe('22_23_eng.csv', '2022-2023')
premier_league_season23_24 = prepare_dataframe('23_24_eng.csv', '2023-2024')
premier_league_season24_25 = prepare_dataframe('24_25_eng.csv', '2024-2025')

premier_league_data = pd.concat([premier_league_season17_19, premier_league_season18_19, premier_league_season19_20, premier_league_season20_21, premier_league_season21_22, premier_league_season22_23, premier_league_season23_24,premier_league_season24_25])
premier_league_data.fillna(0, inplace = True)
premier_league_data.to_csv('premier_league_data.csv', index = False)

In [4]:
def prepare_dataframe_championship(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/10/{season}/schedule/{season}-Championship-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '10')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

In [7]:
championship_season18_19 = prepare_dataframe_championship('championship_18_19.csv', '2018-2019')
championship_season19_20 = prepare_dataframe_championship('championship_19_20.csv', '2019-2020')
championship_season20_21 = prepare_dataframe_championship('championship_20_21.csv', '2020-2021')
championship_season21_22 = prepare_dataframe_championship('championship_21_22.csv', '2021-2022')
championship_season22_23 = prepare_dataframe_championship('championship_22_23.csv', '2022-2023')
championship_season23_24 = prepare_dataframe_championship('championship_23_24.csv', '2023-2024')
championship_season24_25 = prepare_dataframe_championship('championship_24_25.csv', '2024-2025')

In [8]:
championship_data = pd.concat([championship_season18_19, championship_season19_20, championship_season20_21, championship_season21_22, championship_season22_23, championship_season23_24,championship_season24_25])
championship_data.fillna(0, inplace = True)
championship_data.to_csv('championship_data.csv', index = False)

In [20]:
england_data = pd.concat([premier_league_data, championship_data])

In [22]:
england_data.fillna(0, inplace = True)

In [24]:
england_data.sort_values(['Date'], ascending = True, inplace = True)

In [58]:
england_data.reset_index(inplace= True, drop = True)

In [70]:

# Initialize columns for rivalry history
df['HomeWins'] = 0
df['AwayWins'] = 0
df['Draws'] = 0

# Initialize an empty dictionary to track rivalry histories
rivalry_history = {}

# Iterate through each match in the DataFrame
for index, row in df.iterrows():
    # Identify the home and away teams
    home_team = row.filter(like='HomeTeam_').idxmax().replace('HomeTeam_', '')
    away_team = row.filter(like='AwayTeam_').idxmax().replace('AwayTeam_', '')
    
    # Create a tuple key for the rivalry history in specific home-away order
    rivalry_key = (home_team, away_team)
    
    # Initialize rivalry history for this specific home-away matchup if it doesn't exist
    if rivalry_key not in rivalry_history:
        rivalry_history[rivalry_key] = {'HomeWins': 0, 'AwayWins': 0, 'Draws': 0}
    
    # Record the historical values before this match
    df.at[index, 'HomeWins'] = rivalry_history[rivalry_key]['HomeWins']
    df.at[index, 'AwayWins'] = rivalry_history[rivalry_key]['AwayWins']
    df.at[index, 'Draws'] = rivalry_history[rivalry_key]['Draws']
    
    # Update the rivalry history dictionary including the current match's result
    if row['FTHG'] > row['FTAG']:
        # Home team won
        rivalry_history[rivalry_key]['HomeWins'] += 1
    elif row['FTHG'] < row['FTAG']:
        # Away team won
        rivalry_history[rivalry_key]['AwayWins'] += 1
    else:
        # Draw
        rivalry_history[rivalry_key]['Draws'] += 1


In [71]:
england_data[['HomeTeam_Arsenal', 'AwayTeam_Tottenham','HomeTeam_Tottenham','AwayTeam_Arsenal', 'FTHG', 'FTAG','HomeWins','AwayWins','Draws']].loc[((england_data[['HomeTeam_Arsenal', 'AwayTeam_Tottenham', 'FTHG', 'FTAG','HomeWins','AwayWins','Draws']]['HomeTeam_Arsenal'] == 1) & (england_data[['HomeTeam_Arsenal', 'AwayTeam_Tottenham', 'FTHG', 'FTAG','HomeWins','AwayWins','Draws']]['AwayTeam_Tottenham'] == 1)) | ((england_data[['HomeTeam_Tottenham', 'AwayTeam_Arsenal', 'FTHG', 'FTAG','HomeWins','AwayWins','Draws']]['HomeTeam_Tottenham'] == 1) & (england_data[['HomeTeam_Tottenham', 'AwayTeam_Arsenal', 'FTHG', 'FTAG','HomeWins','AwayWins','Draws']]['AwayTeam_Arsenal'] == 1))]

,HomeTeam_Arsenal,AwayTeam_Tottenham,HomeTeam_Tottenham,AwayTeam_Arsenal,FTHG,FTAG,HomeWins,AwayWins,Draws
117,1.0,1.0,0.0,0.0,2,0,0,0,0
264,0.0,0.0,1.0,1.0,1,0,0,0,0
755,1.0,1.0,0.0,0.0,4,2,1,0,0
1079,0.0,0.0,1.0,1.0,1,1,1,0,0
1422,1.0,1.0,0.0,0.0,2,2,2,0,0
2172,0.0,0.0,1.0,1.0,2,1,1,0,1
2540,0.0,0.0,1.0,1.0,2,0,2,0,1
2953,1.0,1.0,0.0,0.0,2,1,2,0,1
3342,1.0,1.0,0.0,0.0,3,1,3,0,1
4085,0.0,0.0,1.0,1.0,3,0,3,0,1
